In [12]:
import torch
dicriminator_weights = 'deberta/deberta-v3-large/pytorch_model.bin'
generator_weights = 'deberta/deberta-v3-large/pytorch_model.generator.bin'
dicriminator = torch.load(dicriminator_weights, map_location='cpu')
generator = torch.load(generator_weights, map_location='cpu')
if 'deberta.embeddings.word_embeddings._weight' not in dicriminator:
    dicriminator['deberta.embeddings.word_embeddings._weight'] = (dicriminator['deberta.embeddings.word_embeddings.weight'] - generator['deberta.embeddings.word_embeddings.weight']).clone()
    dicriminator['deberta.embeddings.position_embeddings._weight'] = (dicriminator['deberta.embeddings.position_embeddings.weight'] - generator['deberta.embeddings.position_embeddings.weight']).clone()
torch.save(dicriminator, "deberta/deberta-v3-large/pytorch_model.fix.bin")

In [10]:
import torch
dicriminator_weights = 'deberta/mdeberta-v3-base/pytorch_model.bin'
generator_weights = 'deberta/mdeberta-v3-base/pytorch_model.generator.bin'

original_vocab_size = 251000
target_vocab_size = 128100

discriminator = torch.load(dicriminator_weights, map_location='cpu')
generator = torch.load(generator_weights, map_location='cpu')

for state_dict in [discriminator, generator]:
    for key in list(state_dict.keys()):
        if state_dict[key].shape[0] == original_vocab_size:
            print('before', key, state_dict[key].shape)
            state_dict[key] = state_dict[key].split(target_vocab_size)[0].clone()
            print('after', key, state_dict[key].shape)

torch.save(discriminator, "deberta/mdeberta-v3-base/pytorch_model.reshape.bin")
torch.save(generator, "deberta/mdeberta-v3-base/pytorch_model.reshape.generator.bin")

before deberta.embeddings.word_embeddings.weight torch.Size([251000, 768])
after deberta.embeddings.word_embeddings.weight torch.Size([128100, 768])
before lm_predictions.lm_head.bias torch.Size([251000])
after lm_predictions.lm_head.bias torch.Size([128100])
before deberta.embeddings.word_embeddings._weight torch.Size([251000, 768])
after deberta.embeddings.word_embeddings._weight torch.Size([128100, 768])
before lm_predictions.lm_head.bias torch.Size([251000])
after lm_predictions.lm_head.bias torch.Size([128100])
before deberta.embeddings.word_embeddings.weight torch.Size([251000, 768])
after deberta.embeddings.word_embeddings.weight torch.Size([128100, 768])


In [11]:
!ls -lsh deberta/mdeberta-v3-base

total 3.4G
1.3G -rw-r--r-- 1 root root 1.3G Apr  6 02:32 pytorch_model.bin
906M -rw-r--r-- 1 root root 906M Apr  6 02:32 pytorch_model.generator.bin
731M -rw-r--r-- 1 root root 731M Jun  1 23:54 pytorch_model.reshape.bin
546M -rw-r--r-- 1 root root 546M Jun  1 23:54 pytorch_model.reshape.generator.bin


In [10]:
discriminator = torch.load("deberta/mdeberta-v3-base/pytorch_model.reshape.bin", map_location='cpu')
discriminator['deberta.embeddings.word_embeddings.weight']

tensor([[ 0.0187, -0.0047, -0.0024,  ..., -0.0198,  0.0154,  0.0246],
        [-0.0253, -0.0084, -0.0109,  ..., -0.0201, -0.0056, -0.0047],
        [-0.0233, -0.0060, -0.0109,  ..., -0.0165, -0.0049, -0.0019],
        ...,
        [-0.2800, -0.0232, -0.0640,  ...,  0.1013,  0.0841,  0.1001],
        [-0.3164,  0.0015, -0.2363,  ..., -0.0097,  0.0348,  0.2197],
        [-0.2710, -0.0572, -0.0813,  ...,  0.0056,  0.1879, -0.0383]],
       dtype=torch.float16)

In [23]:
!sha1sum "deberta/mdeberta-v3-base/pytorch_model.bin"

2d4fb21726f2c7e7a5515828de1fa5ccbbde9795  deberta/mdeberta-v3-base/pytorch_model.bin


In [22]:
!sha1sum "deberta/mdeberta-v3-base/pytorch_model.reshape.bin"

6c70605d29ff9af82911bc628efd40f04d5a9002  deberta/mdeberta-v3-base/pytorch_model.reshape.bin


In [5]:
import torch
original = torch.load('deberta/models/deberta-v3-large-fix-v4-2/discriminator/pytorch.model-015000.bin', map_location='cpu')
original_g = torch.load('deberta/models/deberta-v3-large-fix-v4-2/generator/pytorch.model-015000.bin', map_location='cpu')

#[(k, original[k].shape, original[k]) for k in original if 'deberta.embeddings' in k]
print('discriminator.embeddings.word_embeddings.weight', original['deberta.embeddings.word_embeddings.weight'])
print('discriminator.embeddings.word_embeddings._weight', original['deberta.embeddings.word_embeddings._weight'])

print('discriminator.embeddings.position_embeddings.weight', original['deberta.embeddings.position_embeddings.weight'])
print('discriminator.embeddings.position_embeddings._weight', original['deberta.embeddings.position_embeddings._weight'])

print('generator.embeddings.word_embeddings.weight', original_g['deberta.embeddings.word_embeddings.weight'])
print('generator.embeddings.position_embeddings.weight', original_g['deberta.embeddings.position_embeddings.weight'])

print('should equal to discriminator.embeddings.word_embeddings._weight', original['deberta.embeddings.word_embeddings.weight'] - original_g['deberta.embeddings.word_embeddings.weight'])
print('should equal to discriminator.embeddings.position_embeddings._weight', original['deberta.embeddings.position_embeddings.weight'] - original_g['deberta.embeddings.position_embeddings.weight'])

print('should equal to discriminator.embeddings.word_embeddings.weight', original['deberta.embeddings.word_embeddings._weight'] + original_g['deberta.embeddings.word_embeddings.weight'])
print('should equal to discriminator.embeddings.position_embeddings.weight', original['deberta.embeddings.position_embeddings._weight'] + original_g['deberta.embeddings.position_embeddings.weight'])

discriminator.embeddings.word_embeddings.weight tensor([[-0.0501, -0.0456, -0.0493,  ..., -0.0525, -0.0520, -0.0367],
        [-0.0024, -0.0065,  0.0099,  ..., -0.0135, -0.0091, -0.0034],
        [-0.0002, -0.0050,  0.0105,  ..., -0.0146, -0.0082, -0.0082],
        ...,
        [-0.0497, -0.0591, -0.0370,  ..., -0.0590, -0.0502, -0.0357],
        [-0.0446, -0.0495, -0.0461,  ..., -0.0547, -0.0564, -0.0456],
        [-0.0398, -0.0425, -0.0417,  ..., -0.0585, -0.0485, -0.0334]],
       dtype=torch.float16)
discriminator.embeddings.word_embeddings._weight tensor([[-0.0690, -0.0634, -0.0508,  ..., -0.0562, -0.0447, -0.0366],
        [-0.0070,  0.0010,  0.0031,  ..., -0.0152, -0.0202, -0.0076],
        [-0.0196,  0.0115,  0.0088,  ..., -0.0115, -0.0091, -0.0049],
        ...,
        [-0.0609, -0.0787, -0.0340,  ..., -0.0591, -0.0501, -0.0352],
        [-0.0704, -0.0728, -0.0488,  ..., -0.0565, -0.0470, -0.0415],
        [-0.0572, -0.0560, -0.0423,  ..., -0.0613, -0.0490, -0.0315]],
       

In [13]:
!rm -rf  models/deberta-v3-large-mental-health-v2
!git lfs install
!git clone https://huggingface.co/microsoft/deberta-v3-large models/deberta-v3-large-mental-health-v2
!rm models/deberta-v3-large-mental-health-v2/tf_model.h5

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 'models/deberta-v3-large-mental-health-v2'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 75 (delta 0), reused 0 (delta 0), pack-reused 71
Unpacking objects: 100% (75/75), 10.29 KiB | 658.00 KiB/s, done.
Filtering content: 100% (4/4), 2.96 GiB | 4.33 MiB/s, done.


In [9]:
import os
import torch

modelpath = "deberta/models/deberta-v3-large-fix-v4-2/{}/pytorch.model-{}.bin"
checkpoint = "025000"
output_path = 'models/deberta-v3-large-mental-health-v2'

discriminator = torch.load(modelpath.format('discriminator', checkpoint), map_location='cpu')
del discriminator['deberta.embeddings.word_embeddings._weight']
del discriminator['deberta.embeddings.position_embeddings._weight']
torch.save(discriminator, os.path.join(output_path, 'pytorch_model.bin'))
generator = torch.load(modelpath.format('generator', checkpoint), map_location='cpu')
torch.save(generator, os.path.join(output_path, 'pytorch_model.generator.bin'))

In [91]:
#!cp deberta/models/deberta-v3-large-es-v3/discriminator/pytorch.model-003000.bin models/deberta-v3-large-mental-health-v2/pytorch_model.bin
#!cp deberta/models/deberta-v3-large-es-v3/generator/pytorch.model-003000.bin models/deberta-v3-large-mental-health-v2/pytorch_model.generator.bin

In [3]:
from transformers import AutoModel, AutoTokenizer
path = 'roberta-mental-health'
#AutoModel.from_pretrained('microsoft/deberta-v3-large').save_pretrained('models/deberta-v3-large-mental-health-v2')
#AutoTokenizer.from_pretrained('microsoft/deberta-v3-large').save_pretrained('models/deberta-v3-large-mental-health-v2')

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
!rm models/deberta-v3-large-mental-health-v2/tf_model.h5

In [2]:
import torch

discriminator = torch.load('deberta/models/deberta-v3-large-v2/discriminator/pytorch.model-005000.bin', map_location='cpu')

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [58]:
d_k = [k for k in discriminator.keys() if 'deberta.embeddings' in k]
d_k

['deberta.embeddings.word_embeddings._weight',
 'deberta.embeddings.word_embeddings.weight',
 'deberta.embeddings.position_embeddings._weight',
 'deberta.embeddings.position_embeddings.weight',
 'deberta.embeddings.LayerNorm.weight',
 'deberta.embeddings.LayerNorm.bias']

In [24]:
original = torch.load('deberta/models/deberta-v3-large-es-v3/discriminator/pytorch.model-003000.bin', map_location='cpu')
[(k, original[k].shape, original[k]) for k in original if 'deberta.embeddings' in k]

[('deberta.embeddings.word_embeddings.weight',
  torch.Size([128100, 1024]),
  tensor([[ 1.8890e-02,  1.7776e-02,  1.4286e-03,  ...,  3.7632e-03,
           -7.3776e-03, -9.8228e-05],
          [ 4.7646e-03, -7.3166e-03,  8.7433e-03,  ...,  1.0443e-03,
            1.1230e-02,  2.2469e-03],
          [ 1.8829e-02, -1.5381e-02,  9.7942e-04,  ..., -2.5730e-03,
            1.5574e-03, -1.1778e-03],
          ...,
          [ 1.1292e-02,  1.9653e-02, -3.0918e-03,  ...,  1.2565e-04,
           -1.7440e-04, -5.7507e-04],
          [ 2.5970e-02,  2.3392e-02,  2.6569e-03,  ...,  1.8206e-03,
           -9.4528e-03, -4.1046e-03],
          [ 1.7502e-02,  1.3474e-02,  5.7983e-04,  ...,  2.7714e-03,
            4.3130e-04, -1.9255e-03]], dtype=torch.float16)),
 ('deberta.embeddings.position_embeddings.weight',
  torch.Size([512, 1024]),
  tensor([[-0.0067,  0.0005,  0.0020,  ...,  0.0021,  0.0041, -0.0009],
          [-0.0199, -0.0573, -0.0528,  ..., -0.1196, -0.0335,  0.0053],
          [ 0.0096, 

In [25]:
original = torch.load('deberta/models/deberta-v3-large-es-v3/generator/pytorch.model-003000.bin', map_location='cpu')
[(k, original[k].shape, original[k]) for k in original if 'deberta.embeddings' in k]

[('deberta.embeddings.word_embeddings.weight',
  torch.Size([128100, 1024]),
  tensor([[ 1.8890e-02,  1.7776e-02,  1.4286e-03,  ...,  3.7632e-03,
           -7.3776e-03, -9.8228e-05],
          [ 4.7646e-03, -7.3166e-03,  8.7433e-03,  ...,  1.0443e-03,
            1.1230e-02,  2.2469e-03],
          [ 1.8829e-02, -1.5381e-02,  9.7942e-04,  ..., -2.5730e-03,
            1.5574e-03, -1.1778e-03],
          ...,
          [ 1.1292e-02,  1.9653e-02, -3.0918e-03,  ...,  1.2565e-04,
           -1.7440e-04, -5.7507e-04],
          [ 2.5970e-02,  2.3392e-02,  2.6569e-03,  ...,  1.8206e-03,
           -9.4528e-03, -4.1046e-03],
          [ 1.7502e-02,  1.3474e-02,  5.7983e-04,  ...,  2.7714e-03,
            4.3130e-04, -1.9255e-03]], dtype=torch.float16)),
 ('deberta.embeddings.position_embeddings.weight',
  torch.Size([512, 1024]),
  tensor([[-0.0067,  0.0005,  0.0020,  ...,  0.0021,  0.0041, -0.0009],
          [-0.0199, -0.0573, -0.0528,  ..., -0.1196, -0.0335,  0.0053],
          [ 0.0096, 

In [20]:
original = torch.load('deberta/deberta-v3-large/pytorch_model.bin', map_location='cpu')
[(k, original[k].shape, original[k]) for k in original if 'deberta.embeddings' in k]

[('deberta.embeddings.word_embeddings.weight',
  torch.Size([128100, 1024]),
  tensor([[-0.0504, -0.0459, -0.0496,  ..., -0.0528, -0.0523, -0.0369],
          [-0.0017, -0.0081,  0.0130,  ..., -0.0137, -0.0064, -0.0035],
          [-0.0034, -0.0082,  0.0118,  ..., -0.0140, -0.0061, -0.0045],
          ...,
          [-0.0500, -0.0595, -0.0373,  ..., -0.0593, -0.0505, -0.0359],
          [-0.0448, -0.0498, -0.0464,  ..., -0.0550, -0.0567, -0.0458],
          [-0.0400, -0.0428, -0.0420,  ..., -0.0588, -0.0488, -0.0336]],
         dtype=torch.float16)),
 ('deberta.embeddings.position_embeddings.weight',
  torch.Size([512, 1024]),
  tensor([[ 5.1737e-05,  5.0843e-05, -6.3777e-06,  ...,  2.0266e-06,
            7.5102e-06, -4.6372e-05],
          [ 1.0138e-01,  1.2589e-02, -8.4656e-02,  ..., -1.9812e-01,
            9.2041e-02,  6.9214e-02],
          [ 9.3933e-02,  1.7166e-02, -1.1780e-02,  ...,  1.9485e-02,
            8.0505e-02,  6.5735e-02],
          ...,
          [-2.7542e-03, -1.96

In [19]:
original_g = torch.load('deberta/deberta-v3-large/pytorch_model.generator.bin', map_location='cpu')
[(k, original_g[k].shape, original_g[k]) for k in original_g if 'deberta.embeddings' in k]

[('deberta.embeddings.word_embeddings.weight',
  torch.Size([128100, 1024]),
  tensor([[ 1.8906e-02,  1.7792e-02,  1.4267e-03,  ...,  3.7670e-03,
           -7.3891e-03, -9.7692e-05],
          [ 4.4289e-03, -7.9422e-03,  8.4000e-03,  ...,  8.1301e-04,
            1.0284e-02,  3.8700e-03],
          [ 1.6312e-02, -1.6815e-02,  2.1744e-03,  ..., -2.0256e-03,
            2.1210e-03,  1.0052e-03],
          ...,
          [ 1.1299e-02,  1.9669e-02, -3.0994e-03,  ...,  1.2505e-04,
           -1.7583e-04, -5.7459e-04],
          [ 2.6001e-02,  2.3422e-02,  2.6569e-03,  ...,  1.8225e-03,
           -9.4681e-03, -4.1084e-03],
          [ 1.7517e-02,  1.3489e-02,  5.7650e-04,  ...,  2.7733e-03,
            4.3035e-04, -1.9264e-03]], dtype=torch.float16)),
 ('deberta.embeddings.position_embeddings.weight',
  torch.Size([512, 1024]),
  tensor([[-0.0067,  0.0005,  0.0020,  ...,  0.0021,  0.0041, -0.0009],
          [-0.0204, -0.0512, -0.0542,  ..., -0.1222, -0.0332,  0.0087],
          [ 0.0121, 

In [32]:
original = torch.load('deberta/mdeberta-v3-base/pytorch_model.bin', map_location='cpu')
#[(k, original[k].shape, original[k]) for k in original if 'deberta.embeddings' in k]
print('deberta.embeddings.word_embeddings.weight', original['deberta.embeddings.word_embeddings.weight'])
print('deberta.embeddings.word_embeddings._weight', original['deberta.embeddings.word_embeddings._weight'])

print('deberta.embeddings.position_embeddings.weight', original['deberta.embeddings.position_embeddings.weight'])
print('deberta.embeddings.position_embeddings._weight', original['deberta.embeddings.position_embeddings._weight'])

deberta.embeddings.word_embeddings.weight tensor([[ 0.0187, -0.0047, -0.0024,  ..., -0.0198,  0.0154,  0.0246],
        [-0.0253, -0.0084, -0.0109,  ..., -0.0201, -0.0056, -0.0047],
        [-0.0233, -0.0060, -0.0109,  ..., -0.0165, -0.0049, -0.0019],
        ...,
        [ 0.0157, -0.0065, -0.0094,  ..., -0.0145,  0.0058,  0.0148],
        [ 0.0123, -0.0027, -0.0048,  ..., -0.0144,  0.0147,  0.0226],
        [ 0.0246,  0.0043, -0.0025,  ..., -0.0212,  0.0232,  0.0247]],
       dtype=torch.float16)
deberta.embeddings.word_embeddings._weight tensor([[ 0.0099,  0.0036,  0.0137,  ..., -0.0282,  0.0195,  0.0024],
        [-0.0341, -0.0494, -0.0281,  ..., -0.0823, -0.0181, -0.0084],
        [-0.0222,  0.0148, -0.0013,  ..., -0.0286, -0.0090, -0.0076],
        ...,
        [ 0.0329,  0.0123,  0.0083,  ..., -0.0286,  0.0269,  0.0105],
        [ 0.0257,  0.0163,  0.0057,  ..., -0.0081, -0.0016,  0.0111],
        [ 0.0391,  0.0337,  0.0045,  ..., -0.0149,  0.0448,  0.0351]])
deberta.embeddings.

In [33]:
original_g = torch.load('deberta/mdeberta-v3-base/pytorch_model.generator.bin', map_location='cpu')
#[(k, original_g[k].shape, original_g[k]) for k in original_g if 'deberta.embeddings' in k]
print('deberta.embeddings.word_embeddings.weight', original_g['deberta.embeddings.word_embeddings.weight'])
print('deberta.embeddings.position_embeddings.weight', original_g['deberta.embeddings.position_embeddings.weight'])

deberta.embeddings.word_embeddings.weight tensor([[ 0.0088, -0.0082, -0.0161,  ...,  0.0084, -0.0041,  0.0222],
        [ 0.0087,  0.0409,  0.0172,  ...,  0.0622,  0.0125,  0.0036],
        [-0.0011, -0.0209, -0.0095,  ...,  0.0122,  0.0040,  0.0058],
        ...,
        [-0.0172, -0.0188, -0.0176,  ...,  0.0141, -0.0211,  0.0043],
        [-0.0133, -0.0190, -0.0105,  ..., -0.0062,  0.0163,  0.0115],
        [-0.0145, -0.0294, -0.0071,  ..., -0.0063, -0.0215, -0.0104]])
deberta.embeddings.position_embeddings.weight tensor([[ 1.4325e-02, -3.1255e-02, -5.6232e-03,  ...,  1.8072e-02,
         -1.1372e-02, -1.8335e-02],
        [ 1.4725e-02, -3.0408e-02,  8.2078e-03,  ..., -3.3722e-02,
          2.7028e-02, -1.8940e-02],
        [-1.1823e-02, -5.4432e-03,  1.9808e-02,  ...,  8.3164e-03,
         -1.4451e-03,  7.1966e-03],
        ...,
        [-1.2880e-03,  9.6206e-05,  7.0541e-03,  ...,  4.0525e-03,
         -6.0144e-03,  1.6610e-02],
        [ 1.6347e-02, -1.1903e-02,  2.3155e-02,  ...,

In [34]:
print('calx', original['deberta.embeddings.word_embeddings.weight'] - original_g['deberta.embeddings.word_embeddings.weight'])
print('calx', original['deberta.embeddings.position_embeddings.weight'] - original_g['deberta.embeddings.position_embeddings.weight'])

calx tensor([[ 0.0099,  0.0036,  0.0137,  ..., -0.0282,  0.0195,  0.0024],
        [-0.0341, -0.0494, -0.0281,  ..., -0.0823, -0.0181, -0.0084],
        [-0.0222,  0.0148, -0.0013,  ..., -0.0286, -0.0090, -0.0076],
        ...,
        [ 0.0329,  0.0123,  0.0083,  ..., -0.0286,  0.0269,  0.0105],
        [ 0.0257,  0.0163,  0.0057,  ..., -0.0081, -0.0016,  0.0111],
        [ 0.0391,  0.0337,  0.0045,  ..., -0.0149,  0.0448,  0.0351]])
calx tensor([[-0.0228,  0.0326,  0.0095,  ..., -0.0077,  0.0206,  0.0182],
        [-0.0818,  0.1288,  0.0435,  ..., -0.0429, -0.0476,  0.1203],
        [ 0.0064,  0.0594, -0.0121,  ..., -0.0809, -0.0295,  0.0544],
        ...,
        [-0.0018,  0.0096,  0.0228,  ..., -0.0247, -0.0489, -0.0758],
        [-0.0328,  0.0074,  0.0187,  ..., -0.0251, -0.0750, -0.0602],
        [-0.0044,  0.0079, -0.0119,  ...,  0.0212,  0.0060, -0.0074]])


In [13]:
original = torch.load('deberta/mdeberta-v3-base/pytorch_model.generator.bin', map_location='cpu')
[(k, original[k].shape) for k in original if 'deberta.embeddings' in k]

[('deberta.embeddings.word_embeddings.weight', torch.Size([251000, 768])),
 ('deberta.embeddings.position_embeddings.weight', torch.Size([512, 768])),
 ('deberta.embeddings.LayerNorm.weight', torch.Size([768])),
 ('deberta.embeddings.LayerNorm.bias', torch.Size([768]))]

['deberta.embeddings.word_embeddings.weight',
 'deberta.embeddings.position_embeddings.weight',
 'deberta.embeddings.LayerNorm.weight',
 'deberta.embeddings.LayerNorm.bias']

In [85]:
len(discriminator)

399

In [81]:
o_k = [k for k in original.keys() if 'deberta.embeddings' in k]
o_k

['deberta.embeddings.word_embeddings.weight',
 'deberta.embeddings.position_embeddings.weight',
 'deberta.embeddings.LayerNorm.weight',
 'deberta.embeddings.LayerNorm.bias',
 'deberta.embeddings.word_embeddings._weight',
 'deberta.embeddings.position_embeddings._weight']

In [82]:
for k in d_k:
    if k in original:
        print('original', k, original[k])
    if k in discriminator:
        print('discriminator', k, discriminator[k])
    print()

original deberta.embeddings.word_embeddings._weight tensor([[ 0.0099,  0.0036,  0.0137,  ..., -0.0282,  0.0195,  0.0024],
        [-0.0341, -0.0494, -0.0281,  ..., -0.0823, -0.0181, -0.0084],
        [-0.0222,  0.0148, -0.0013,  ..., -0.0286, -0.0090, -0.0076],
        ...,
        [ 0.0329,  0.0123,  0.0083,  ..., -0.0286,  0.0269,  0.0105],
        [ 0.0257,  0.0163,  0.0057,  ..., -0.0081, -0.0016,  0.0111],
        [ 0.0391,  0.0337,  0.0045,  ..., -0.0149,  0.0448,  0.0351]])
discriminator deberta.embeddings.word_embeddings._weight tensor([[-0.0503, -0.0458, -0.0495,  ..., -0.0526, -0.0522, -0.0368],
        [-0.0002, -0.0090,  0.0163,  ..., -0.0106, -0.0101, -0.0032],
        [-0.0028, -0.0092,  0.0134,  ..., -0.0107, -0.0090, -0.0089],
        ...,
        [-0.0499, -0.0594, -0.0372,  ..., -0.0592, -0.0504, -0.0358],
        [-0.0447, -0.0497, -0.0463,  ..., -0.0549, -0.0566, -0.0457],
        [-0.0399, -0.0427, -0.0419,  ..., -0.0587, -0.0487, -0.0335]],
       dtype=torch.floa

In [54]:
original

{'deberta.embeddings.word_embeddings.weight': tensor([[-0.0504, -0.0459, -0.0496,  ..., -0.0528, -0.0523, -0.0369],
         [-0.0017, -0.0081,  0.0130,  ..., -0.0137, -0.0064, -0.0035],
         [-0.0034, -0.0082,  0.0118,  ..., -0.0140, -0.0061, -0.0045],
         ...,
         [-0.0500, -0.0595, -0.0373,  ..., -0.0593, -0.0505, -0.0359],
         [-0.0448, -0.0498, -0.0464,  ..., -0.0550, -0.0567, -0.0458],
         [-0.0400, -0.0428, -0.0420,  ..., -0.0588, -0.0488, -0.0336]],
        dtype=torch.float16),
 'deberta.embeddings.position_embeddings.weight': tensor([[ 5.1737e-05,  5.0843e-05, -6.3777e-06,  ...,  2.0266e-06,
           7.5102e-06, -4.6372e-05],
         [ 1.0138e-01,  1.2589e-02, -8.4656e-02,  ..., -1.9812e-01,
           9.2041e-02,  6.9214e-02],
         [ 9.3933e-02,  1.7166e-02, -1.1780e-02,  ...,  1.9485e-02,
           8.0505e-02,  6.5735e-02],
         ...,
         [-2.7542e-03, -1.9638e-02,  6.8909e-02,  ..., -2.5711e-03,
           3.5614e-02,  1.6724e-02],
 

In [22]:
discriminator['deberta.embeddings.word_embeddings._weight'] = discriminator['deberta.embeddings.word_embeddings.weight']
discriminator['deberta.embeddings.position_embeddings._weight'] = discriminator['deberta.embeddings.position_embeddings.weight']

In [83]:
AutoModel.from_pretrained('microsoft/deberta-v3-large')

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(128100, 1024, padding_idx=0)
    (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
    (dropout): StableDropout()
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0): DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (pos_dropout): StableDropout()
            (dropout): StableDropout()
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
            (dropout): StableDropout()
          )
        )
        (interme

In [66]:
AutoModel.from_pretrained('models/deberta-v3-large-mental-health-v2').save_pretrained('models/test')

Some weights of the model checkpoint at models/deberta-v3-large-mental-health-v2 were not used when initializing DebertaV2Model: ['mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'deberta.embeddings.word_embeddings._weight', 'mask_predictions.dense.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [68]:
import torch

new = torch.load('models/test/pytorch_model.bin', map_location='cpu')

In [70]:
n_k = [k for k in original.keys() if 'deberta.embeddings' in k]
n_k

['deberta.embeddings.word_embeddings.weight',
 'deberta.embeddings.position_embeddings.weight',
 'deberta.embeddings.LayerNorm.weight',
 'deberta.embeddings.LayerNorm.bias']

In [6]:
!mkdir models/roberta-large-mental-health-v1
!cp deproberta/trained_models/deproberta_v2/checkpoint-151519-epoch-1/* models/roberta-large-mental-health-v1

In [ ]:
!cp trained_models/deproberta_v2/checkpoint-151519-epoch-1/pytorch_model.bin pytorch_model.bin

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs/